In [2]:
# Import libraries
import requests
import google.generativeai as genai
from dotenv import load_dotenv
import os
import pandas as pd
import tabulate
import googlemaps
import itertools
import numpy as np
from scipy.spatial import distance_matrix
from datetime import datetime

# Load environment variables
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
genai_api_key = os.getenv("GENAI_API_KEY")

# Set up Google Maps client
gmaps = googlemaps.Client(key=google_api_key)

# Set up Generative AI model
genai.configure(api_key=genai_api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
prompt = "Only answer with a range of numbers (e.g. 1-2, 4-6). What is the ideal number of days for a relaxing road trip from Denver to Las Vegas, including leisure stops along the way for sightseeing and unwinding?"

ai_response = model.generate_content(prompt)
print(ai_response.text)

In [ ]:
def main():
    
    while True:
        start_location = input("Enter start location (or 'q' to quit): ").strip()
        if start_location.lower() == 'q':
            return
            
        end_location = input("Enter end location (or 'q' to quit): ").strip()
        if end_location.lower() == 'q':
            return
            
        if not start_location or not end_location:
            print("\nBoth start and end locations are required.")
            continue
            
        # Validate locations using trip_length_recommendation function
        try:
            recommended_days = trip_length_recommendation(start_location, end_location)
            print(f"\nRecommended trip duration: {recommended_days[0]}-{recommended_days[-1]} days")
        except Exception as e:
            print(f"\nError: Could not validate locations. Please try again.")
            print("Make sure to enter valid city names (e.g., 'City, State')")

        chosen_days = None
        while chosen_days is None:
            try:
                chosen_days = int(input(f"Choose the number of days for your trip ({recommended_days[0]}-{recommended_days[-1]}): ").strip())
                if chosen_days not in recommended_days:
                    print(f"Please choose a number within the recommended range ({recommended_days[0]}-{recommended_days[-1]}).")
                    chosen_days = None
            except ValueError:
                print("Invalid input. Please enter a number.")

        print(f"\nYou chose a trip duration of {chosen_days} days.")

        # Get user preferences
        interests = input("Enter your interests (e.g., 'nature, history, food'): ").strip().split(", ")
        wanted_stops = input("Enter the stops that must be in the trip: ").strip()

        stops = stops_recommendation(start_location, end_location, interests, wanted_stops, chosen_days)

        route_optimization(start_location, end_location, stops)
        break

main()

In [7]:
def trip_length_recommendation(start, end):
    if start and end:
        prompt = f"Only answer with a range of numbers (e.g. 1-2, 4-6). What is the ideal number of days for a relaxing road trip from {start} to {end}, including leisure stops along the way for sightseeing and unwinding?"

        ai_response = model.generate_content(prompt)
        range_str = ai_response.text
        
        start, end = map(int, range_str.split('-'))

        return list(range(start, end + 1))

In [9]:
def stops_recommendation(start, end, interests, wanted_stops, trip_days):

    prompt = f"Help me pick destinations that I should spend time at during my {trip_days} long road trip from {start} to {end}. I am interested in {', '.join(interests)}. I want to make {wanted_stops} stops along the way. Please provide a list of recommended stops that align with these interests, ensuring they are spaced out appropriately for the trip duration. Join any stops that are withing 30 miles of each other and do not include the end point. List the stops concisely, separated by semicolons, with no additional descriptions."

    ai_response = model.generate_content(prompt)
    stops = [stop.strip() for stop in ai_response.text.split(";") if stop.strip()]

    print(f'These are the stops suggested by AI:')
    for i, stop in enumerate(stops, 1):
        print(f'{i}. {stop}')
    
    if not stops:
        return "No stops suggested by the AI."
        
    # Ask user if they want to modify stops
    while True:
        modify = input("\nWould you like to modify the stops? (yes/no): ").lower()
        if modify == 'yes':
            action = input("Would you like to add or remove stops? (add/remove): ").lower()
            
            if action == 'add':
                add = input("Manually enter stops to add or ask AI fro suggestions (manual/ai): ").lower()
                if add == 'manual':
                    new_stop = input("Enter the name of the stop to add: ")

                elif add == "ai":
                    prompt = f"Give me more road trip destinations that I should go to considering my interests: {', '.join(interests)}. The trip is from {start} to {end} and is {trip_days} long. Do not include stops that you have already suggested: {', '.join(stops)}. List the stops concisely, separated by semicolons, with no additional descriptions." 
                    ai_response = model.generate_content(prompt)
                    new_stop = [stop.strip() for stop in ai_response.text.split(";") if stop.strip()]
                    print("\nAI suggested these additional stops:")
                    for i, stop in enumerate(new_stop, 1):
                        print(f"{i}. {stop}")

                while True:
                    try:
                        choices = input("\nEnter the numbers of stops you want to add separated by comma(e.g., 1,3,4): ")
                        indices = [int(x.strip()) - 1 for x in choices.split(',')]
                        if all(0 <= i < len(new_stop) for i in indices):
                            selected_stops = [new_stop[i] for i in indices]
                            stops.extend(selected_stops)
                            break
                        else:
                            print("Invalid number(s). Please try again.")

                    except ValueError:
                        print("Please enter valid numbers separated by commas.")
            elif action == 'remove':
                remove_num = int(input("Enter the number of the stop to remove (1-{}): "))
                if 1 <= remove_num <= len(stops):
                    removed_stop = stops.pop(remove_num - 1)
                    print(f"Removed: {removed_stop}")
                else:
                    print("Invalid stop number")
            
            print("\nUpdated stops:")
            for i, stop in enumerate(stops, 1):
                 print(f"{i}. {stop}")
        elif modify == 'no':
            break
        else:
            print("Please enter 'yes' or 'no'")

    return stops

In [11]:
def route_optimization(start, end, stops):
    try:
        directions_result = gmaps.directions(
            start, 
            end, 
            waypoints=stops, 
            mode="driving", 
            optimize_waypoints=True
            )
        
        if directions_result and len(directions_result) > 0:
            optimized_route = directions_result[0]
            optimized_waypoints = optimized_route.get('waypoint_order')

            print(f"Original Waypoints: {stops}")
            print(f"Optimized Order of Waypoints: { [stops[i] for i in optimized_waypoints]}")

            total_distance = 0
            total_duration = 0
            for leg in optimized_route['legs']:
                total_distance += leg['distance']['value']
                total_duration += leg['duration']['value']

            print(f"Total Optimized Distance: {total_distance/1000:.2f} km")
            print(f"Total Optimized Duration: {total_duration/60:.2f} minutes")

            for i, leg in enumerate(optimized_route['legs']):
                    print(f"\nLeg {i+1}: Start: {leg['start_address']}, End: {leg['end_address']}")
                    for step in leg['steps']:
                        print(f"   {step['html_instructions']}")
        else:
            print(f"Could not find directions for optimized route")

    except googlemaps.exceptions.ApiError as e:
        print(f"Error: {e}")